# Price Analytics & Elasticity Measurement

This notebook performs data cleaning on insurance contracts and claims data, and then estimates the price elasticity of demand using a logistic regression model on customer retention.

Data Source: [Insurance Dataset](https://www.kaggle.com/datasets/kpoviesistphane/insurance-dataset-for-data-engineering-practice)

# 0. Load libraries

In [1]:
import pandas as pd
import numpy as np
import re
import plotly.express as px
import plotly.graph_objects as go
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
import os

# Set display options
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

# 1. Data Loading

In [2]:
try:
    # Adjust paths if running from root or Analysis dir
    if os.path.exists("data/claims.csv"):
        claims = pd.read_csv("data/claims.csv")
        contracts = pd.read_csv("data/contracts.csv")
    elif os.path.exists("Analysis/data/claims.csv"):
        claims = pd.read_csv("Analysis/data/claims.csv")
        contracts = pd.read_csv("Analysis/data/contracts.csv")
    else:
        # Fallback for absolute path usage potential
        claims = pd.read_csv(
            r"c:\Users\soumy\OneDrive\Coding\python_scripts\Price-Analytics\Analysis\data\claims.csv"
        )
        contracts = pd.read_csv(
            r"c:\Users\soumy\OneDrive\Coding\python_scripts\Price-Analytics\Analysis\data\contracts.csv"
        )

    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: Data files not found.")
    exit()

Data loaded successfully.


# 2. Data Cleaning Functions

In [3]:
def vectorized_parse_date(series):
    """Vectorized date parsing trying multiple formats in sequence."""
    # Standardize input
    s = series.astype(str).str.strip()
    
    # Initialize with first format
    dates = pd.to_datetime(s, format="%Y-%m-%d", errors="coerce")
    
    # Fallback formats
    formats = ["%d/%m/%Y", "%m/%d/%Y", "%d-%m-%Y"]
    for fmt in formats:
        dates = dates.fillna(pd.to_datetime(s, format=fmt, errors="coerce"))
    
    # Final flexible fallback (slowest, only runs on remaining NaNs)
    mask = dates.isna() & (s != "nan") & (s != "") & (s != "None")
    if mask.any():
        dates.loc[mask] = pd.to_datetime(s[mask], dayfirst=True, errors="coerce")
        
    return dates


# 3. Applying Cleaning Logic

In [4]:
# Contracts
contracts["start_date_clean"] = vectorized_parse_date(contracts["start_date"])
contracts["end_date_clean"] = vectorized_parse_date(contracts["end_date"])

contracts["annual_premium_clean"] = pd.to_numeric(
    contracts["annual_premium"].astype(str).str.replace(r"[^0-9.]", "", regex=True),
    errors="coerce"
)

g_lower = contracts["gender"].astype(str).str.strip().str.lower()
contracts["gender_clean"] = np.select(
    [g_lower.isin(["m", "male"]), g_lower.isin(["f", "female"])],
    ["M", "F"],
    default="Unknown"
)

contracts["age_missing"] = contracts["client_age"].isna().astype(int)
contracts["client_age_clean"] = pd.to_numeric(contracts["client_age"], errors="coerce")
contracts["client_age_clean"] = contracts["client_age_clean"].fillna(
    contracts["client_age_clean"].median()
)
contracts["csp_clean"] = contracts["csp"].fillna("Unknown")
contracts["channel_clean"] = contracts["channel"].fillna("Unknown")
contracts["product_clean"] = contracts["product"].fillna("Unknown")

valid_status_mask = contracts["status"].isin(
    ["Active", "Renewed", "Cancelled", "Expired"]
)
contracts_model = contracts[valid_status_mask].copy()
contracts_model["retained"] = (
    contracts_model["status"].isin(["Active", "Renewed"]).astype(int)
)
contracts_model = contracts_model.dropna(subset=["annual_premium_clean"])

# Claims
claims["occurrence_date_clean"] = vectorized_parse_date(claims["occurrence_date"])
claims["declaration_date_clean"] = vectorized_parse_date(claims["declaration_date"])

claims["damage_amount_clean"] = pd.to_numeric(
    claims["damage_amount"].astype(str).str.replace(r"[^0-9.]", "", regex=True),
    errors="coerce"
)
claims["indemnified_amount_clean"] = pd.to_numeric(
    claims["indemnified_amount"].astype(str).str.replace(r"[^0-9.]", "", regex=True),
    errors="coerce"
)

claim_counts = pd.pivot_table(
    claims,
    index="contract_id",
    columns="claim_type",
    values="claim_id",
    aggfunc="count",
    fill_value=0,
)
claim_counts.columns = ["num_claims_" + str(c) for c in claim_counts.columns]
claim_counts = claim_counts.reset_index()

claim_amounts = (
    claims.groupby("contract_id")
    .agg(
        num_claims=("claim_id", "count"),
        total_claim_cost=("indemnified_amount_clean", "sum"),
        total_damage_amount=("damage_amount_clean", "sum"),
    )
    .reset_index()
)

claims_agg = pd.merge(claim_amounts, claim_counts, on="contract_id", how="left")

# 4. Merging Data

In [5]:
df_final = pd.merge(contracts_model, claims_agg, on="contract_id", how="left")
cols_to_fill = ["num_claims", "total_claim_cost", "total_damage_amount"] + [
    c for c in claims_agg.columns if "num_claims_" in c
]
for col in cols_to_fill:
    if col in df_final.columns:
        df_final[col] = df_final[col].fillna(0)

df_final["annual_premium_clean"] = pd.to_numeric(
    df_final["annual_premium_clean"], errors="coerce"
)
df_final["client_age_clean"] = pd.to_numeric(
    df_final["client_age_clean"], errors="coerce"
)
df_final["num_claims"] = pd.to_numeric(df_final["num_claims"], errors="coerce")
df_final["client_age_clean"] = df_final["client_age_clean"].fillna(
    df_final["client_age_clean"].median()
)
df_final = df_final.dropna(subset=["annual_premium_clean"])


# --- Feature Engineering ---
# 1. Tenure
df_final["end_date_filled"] = df_final["end_date_clean"].fillna(df_final["start_date_clean"] + pd.Timedelta(days=365))
df_final["tenure_days"] = (df_final["end_date_filled"] - df_final["start_date_clean"]).dt.days.fillna(0).clip(lower=0)

# 2. Seasonality
df_final["start_month"] = df_final["start_date_clean"].dt.month.fillna(0).astype(int)

# 3. Financial Ratios
df_final["loss_ratio"] = df_final["total_claim_cost"] / (df_final["annual_premium_clean"] + 1e-6)
safe_tenure_years = df_final["tenure_days"].replace(0, 1) / 365.0
df_final["claim_freq_annualized"] = df_final["num_claims"] / safe_tenure_years

# 4. Log Transforms
df_final["log_annual_premium"] = np.log1p(df_final["annual_premium_clean"])
df_final["log_total_damage_amount"] = np.log1p(df_final["total_damage_amount"])

print(f"Final Dataset Shape: {df_final.shape}")

Final Dataset Shape: (12282, 41)


# 5. Elasticity Analysis & Feature Importance

In [6]:
print("\n--- Starting XGBoost RAM/GPU Optimization ---")

# Prepare features
categorical_cols = [
    "risk_zone",
    "gender_clean",
    "channel_clean",
    "csp_clean",
    "product_clean",
]
df_model = pd.get_dummies(df_final, columns=categorical_cols, drop_first=True)

base_vars = [
    "annual_premium_clean", "client_age_clean", "total_damage_amount",
    "tenure_days", "start_month", "loss_ratio", "claim_freq_annualized",
    "log_annual_premium", "log_total_damage_amount"
]
claim_type_vars = [c for c in df_model.columns if "num_claims_" in c]
dummy_vars = [
    c
    for c in df_model.columns
    if any(
        stem in c
        for stem in [
            "risk_zone_",
            "gender_clean_",
            "channel_clean_",
            "csp_clean_",
            "product_clean_",
        ]
    )
]

independent_vars = base_vars + claim_type_vars + dummy_vars

X = df_model[independent_vars]
X = X.astype(float)
y = df_model["retained"].astype(int)

# 1. Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Train Clean: {X_train.shape}, Test Clean: {X_test.shape}")

# 2. Hyperparameter Grid (XGBoost specific)
param_dist = {
    "n_estimators": [100, 200, 300],
    "max_depth": [3, 6, 10, 15],
    "learning_rate": [0.01, 0.05, 0.1, 0.3],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "min_child_weight": [1, 3, 5],
}

# 3. XGBoost Classifier with GPU support
print("Initializing XGBoost with GPU support (device='cuda')...")
xgb = XGBClassifier(
    objective='binary:logistic',
    tree_method='hist',
    device='cuda:0',
    n_jobs=4,
    random_state=42,
    eval_metric='logloss'
)

# 4. Randomized Search
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    scoring="accuracy",
    n_jobs=4, 
    random_state=42,
    verbose=1,
)

print("Tuning hyperparameters on GPU...")
print("Note: For small datasets (~12k rows), GPU utilization % may remain low as training is instantaneous.")
try:
    random_search.fit(X_train, y_train)
except Exception as e:
    print(f"GPU Training failed: {e}. Falling back to CPU.")
    xgb.set_params(device='cpu')
    random_search.estimator = xgb
    random_search.fit(X_train, y_train)

best_xgb = random_search.best_estimator_


print("\n--- Best Hyperparameters ---")
print(random_search.best_params_)

# 5. Evaluation
y_pred = best_xgb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"\nTest Set Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# 6. Feature Importance
print("\n--- Feature Importance (Top 10) ---")
importances = pd.DataFrame(
    {"Feature": X.columns, "Importance": best_xgb.feature_importances_}
).sort_values("Importance", ascending=False)

print(importances.head(10))



--- Starting XGBoost RAM/GPU Optimization ---
Train Clean: (9825, 33), Test Clean: (2457, 33)
Initializing XGBoost with GPU support (device='cuda')...
Tuning hyperparameters on GPU...
Note: For small datasets (~12k rows), GPU utilization % may remain low as training is instantaneous.
Fitting 5 folds for each of 20 candidates, totalling 100 fits

--- Best Hyperparameters ---
{'subsample': 1.0, 'n_estimators': 300, 'min_child_weight': 3, 'max_depth': 6, 'learning_rate': 0.01, 'colsample_bytree': 0.6}

Test Set Accuracy: 0.5258

Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.70      0.60      1259
           1       0.52      0.34      0.41      1198

    accuracy                           0.53      2457
   macro avg       0.52      0.52      0.51      2457
weighted avg       0.52      0.53      0.51      2457


--- Feature Importance (Top 10) ---
                    Feature  Importance
4               start_month    0.049292


c:\Users\soumy\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:774: UserWarning: [00:21:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:62: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


# 6. Visualization

In [7]:
try:
    # 1. Price Elasticity Plot
    df_final["premium_bin"] = pd.qcut(df_final["annual_premium_clean"], q=10)
    retention_by_price = (
        df_final.groupby("premium_bin", observed=False)["retained"].mean().reset_index()
    )
    retention_by_price["price_mid"] = retention_by_price["premium_bin"].apply(
        lambda x: x.mid
    )

    fig1 = px.scatter(
        retention_by_price,
        x="price_mid",
        y="retained",
        title="Retention Rate vs Annual Premium (Deciles)",
        labels={"price_mid": "Annual Premium", "retained": "Retention Rate"},
        trendline="ols",
    )
    fig1.show()

    # 2. Feature Importance Plot
    if "importances" in locals():
        fig2 = px.bar(
            importances.head(15),
            x="Importance",
            y="Feature",
            orientation="h",
            title="Top 15 Drivers of Customer Retention (Random Forest)",
            labels={"Importance": "Importance Score", "Feature": "Factor"},
        )
        fig2.update_layout(yaxis={"categoryorder": "total ascending"})
        fig2.show()

except Exception as e:
    print("Visualization failed:", e)


In [11]:
# --- Feature Selection: Removing Low Importance Features ---
print("\n--- Feature Selection: Removing Low Importance Features ---")

# Identify low importance features
# Using a threshold of 0.01
low_importance_mask = best_xgb.feature_importances_ < 0.03
low_importance_features = X.columns[low_importance_mask]

print(f"Dropping {len(low_importance_features)} features: {list(low_importance_features)}")

X_reduced = X.drop(columns=low_importance_features)
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X_reduced, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Reduced Train: {X_train_r.shape}, Reduced Test: {X_test_r.shape}")

print("Retraining XGBoost on reduced feature set...")
# Reuse best params but re-fit. Note: Reuse the same classifier class and params
best_xgb_reduced = XGBClassifier(**best_xgb.get_params())
best_xgb_reduced.fit(X_train_r, y_train_r)

y_pred_r = best_xgb_reduced.predict(X_test_r)
accuracy_r = accuracy_score(y_test_r, y_pred_r)

print(f"\nOriginal Accuracy: {accuracy:.4f}")
print(f"Reduced Accuracy:  {accuracy_r:.4f}")
print("\nClassification Report (Reduced):")
print(classification_report(y_test_r, y_pred_r))

# Feature Importance (Reduced)
print("\n--- Feature Importance (Reduced) ---")
importances_r = pd.DataFrame(
    {"Feature": X_reduced.columns, "Importance": best_xgb_reduced.feature_importances_}
).sort_values("Importance", ascending=False)

print(importances_r.head(10))



--- Feature Selection: Removing Low Importance Features ---
Dropping 12 features: ['num_claims_Collision', 'num_claims_Fire', 'num_claims_Storm', 'num_claims_Vandalism', 'num_claims_Water_damage', 'risk_zone_Medium', 'gender_clean_M', 'channel_clean_Broker', 'channel_clean_Phone', 'csp_clean_Retired', 'csp_clean_Student', 'product_clean_Home']
Reduced Train: (9825, 21), Reduced Test: (2457, 21)
Retraining XGBoost on reduced feature set...

Original Accuracy: 0.5258
Reduced Accuracy:  0.5311

Classification Report (Reduced):
              precision    recall  f1-score   support

           0       0.53      0.71      0.61      1259
           1       0.53      0.35      0.42      1198

    accuracy                           0.53      2457
   macro avg       0.53      0.53      0.51      2457
weighted avg       0.53      0.53      0.52      2457


--- Feature Importance (Reduced) ---
                    Feature  Importance
4               start_month    0.063812
1          client_age_cl